### Model Training

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds

In [15]:
DATA_DIR = '/content/drive/MyDrive/EarVN1.0 dataset/Images'
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 64

In [16]:
train_val_ds = tf.keras.utils.image_dataset_from_directory(
  DATA_DIR,
  validation_split=0.15,
  label_mode='int',
  subset="training",
  seed=999,
  image_size=(IMG_HEIGHT, IMG_WIDTH),
  batch_size=BATCH_SIZE)

Found 28412 files belonging to 164 classes.
Using 24151 files for training.


In [17]:
test_ds = tf.keras.utils.image_dataset_from_directory(
  DATA_DIR,
  validation_split=0.15,
  label_mode='int',
  subset="validation",
  seed=999,
  image_size=(IMG_HEIGHT, IMG_WIDTH),
  batch_size=BATCH_SIZE)

Found 28412 files belonging to 164 classes.
Using 4261 files for validation.


In [18]:
train_val_batches = tf.data.experimental.cardinality(train_val_ds)
train_val_batches

<tf.Tensor: shape=(), dtype=int64, numpy=378>

In [19]:
TOTAL_TRAIN_VAL = len(train_val_ds)
print(TOTAL_TRAIN_VAL)
train_ds = train_val_ds.take(int(8 * train_val_batches / 10))
val_ds = train_val_ds.skip(int(8 * train_val_batches / 10))
print(len(train_ds))
print(len(val_ds))

378
302
76


In [8]:
train_ds

<_TakeDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [9]:
from tensorflow.keras.callbacks import ModelCheckpoint
mc = ModelCheckpoint(
    filepath='./best_efficientnetv2s_model_epoch={epoch:02d}_acc={val_accuracy:0.4f}.h5',
    monitor="val_accuracy",
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    mode='max',
    save_freq='epoch'
)

In [20]:
from keras.models import load_model

model_path = '/content/drive/MyDrive/Colab Notebooks/best_efficientnetv2s_model_epoch=37_acc=0.9046.h5'

model = load_model(model_path)

In [11]:
results = model.fit(train_ds, validation_data=val_ds, epochs=13, batch_size=64, callbacks=[mc])

Epoch 1/13
302/302 [==============================] - ETA: 0s - loss: 0.0318 - accuracy: 0.9905 
Epoch 1: val_accuracy improved from -inf to 0.89778, saving model to ./best_efficientnetv2s_model_epoch=01_acc=0.8978.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


302/302 [==============================] - 4356s 14s/step - loss: 0.0318 - accuracy: 0.9905 - val_loss: 0.4738 - val_accuracy: 0.8978
Epoch 2/13
302/302 [==============================] - ETA: 0s - loss: 0.0340 - accuracy: 0.9898
Epoch 2: val_accuracy improved from 0.89778 to 0.90214, saving model to ./best_efficientnetv2s_model_epoch=02_acc=0.9021.h5
302/302 [==============================] - 444s 1s/step - loss: 0.0340 - accuracy: 0.9898 - val_loss: 0.5194 - val_accuracy: 0.9021
Epoch 3/13
302/302 [==============================] - ETA: 0s - loss: 0.0344 - accuracy: 0.9889
Epoch 3: val_accuracy did not improve from 0.90214
302/302 [==============================] - 385s 1s/step - loss: 0.0344 - accuracy: 0.9889 - val_loss: 0.5089 - val_accuracy: 0.8963
Epoch 4/13
302/302 [==============================] - ETA: 0s - loss: 0.0362 - accuracy: 0.9889
Epoch 4: val_accuracy did not improve from 0.90214
302/302 [==============================] - 382s 1s/step - loss: 0.0362 - accuracy: 0.988

In [21]:
from keras.models import load_model

model_path = '/content/drive/MyDrive/Colab Notebooks/best_efficientnetv2s_model_epoch=48_acc=0.9061.h5'

model = load_model(model_path)

In [22]:
predictions = []
y_true = []
for img, labels in test_ds:
    predictions.extend(model.predict_on_batch(img))
    y_true.extend(labels.numpy())
y_true = np.array(y_true)

In [23]:
y_pred = np.argmax(predictions, axis=1)

In [24]:
from sklearn.metrics import classification_report

report = classification_report(y_true, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.91      0.84      0.87        50
           1       1.00      0.90      0.95        39
           2       0.67      1.00      0.80        24
           3       0.91      0.94      0.92        31
           4       0.91      0.94      0.93        34
           5       0.89      0.97      0.93        35
           6       0.81      0.96      0.88        23
           7       1.00      1.00      1.00        26
           8       0.94      0.89      0.91        35
           9       0.93      0.96      0.95        28
          10       1.00      0.96      0.98        23
          11       0.91      1.00      0.95        29
          12       0.92      1.00      0.96        24
          13       0.81      1.00      0.89        17
          14       0.97      0.85      0.91        34
          15       0.96      0.96      0.96        28
          16       0.95      0.91      0.93        23
          17       0.92    